# Student Alcohol Consumption

### Introduction:

This time you will download a dataset from the UCI.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import requests

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv).

### Step 3. Assign it to a variable called df.

In [2]:

url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/04_Apply/Students_Alcohol_Consumption/student-mat.csv'


get_url = requests.get(url)


with open('data.csv', 'w', encoding='utf-8') as f:
    
    f.write(get_url.text)
    

df = pd.read_csv('data.csv', header=0, sep=',')

spark = SparkSession.builder\
                    .appName('students')\
                    .getOrCreate()
                    
                    
students = spark.read.csv('data.csv', header=True, sep=',')

25/06/12 14:40:15 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.88 instead (on interface wlo1)
25/06/12 14:40:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 14:40:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 4. For the purpose of this exercise slice the dataframe from 'school' until the 'guardian' column

In [3]:

columnas = students.columns


inicio = columnas.index('school')
final = columnas.index('guardian')

subset_columnas = columnas[inicio:final]


students_subset = students.select(subset_columnas)


In [4]:
df_subset = df.loc[:, 'school':'guardian']
df_subset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother
4,GP,F,16,U,GT3,T,3,3,other,other,home,father
...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother
392,MS,M,21,R,GT3,T,1,1,other,other,course,other
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother


### Step 5. Create a lambda function that will capitalize strings.

In [5]:
from pyspark.sql import functions as F

capitalize_udf = F.udf(lambda x: str(x).capitalize() if x is not None else None, F.StringType())


students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["reason"])).show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+----------------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|col_capitalizada|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+----------------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| at_home| teacher|    course|          Course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| at_home|   other|    course|          Course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| at_home|   other|     other|           Other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  health|services|      home|            Home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   other|   other|      home|            Home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|services|   other|reputation|      Reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   other|   other|      home|            Home|
|    GP|  F| 17|      U|    GT

In [6]:
df['Mjob'].apply(lambda x : str(x).capitalize())

0       At_home
1       At_home
2       At_home
3        Health
4         Other
         ...   
390    Services
391    Services
392       Other
393    Services
394       Other
Name: Mjob, Length: 395, dtype: object

25/06/12 14:40:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 6. Capitalize both Mjob and Fjob

In [28]:
students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["Mjob"])).select(F.col('Mjob')).show()
students_subset.withColumn("col_capitalizada", capitalize_udf(students_subset["Fjob"])).select(F.col('Fjob')).show()


+--------+
|    Mjob|
+--------+
| At_home|
| At_home|
| At_home|
|  Health|
|   Other|
|Services|
|   Other|
|   Other|
|Services|
|   Other|
| Teacher|
|Services|
|  Health|
| Teacher|
|   Other|
|  Health|
|Services|
|   Other|
|Services|
|  Health|
+--------+
only showing top 20 rows

+--------+
|    Fjob|
+--------+
| Teacher|
|   Other|
|   Other|
|Services|
|   Other|
|   Other|
|   Other|
| Teacher|
|   Other|
|   Other|
|  Health|
|   Other|
|Services|
|   Other|
|   Other|
|   Other|
|Services|
|   Other|
|Services|
|   Other|
+--------+
only showing top 20 rows



In [27]:
students_subset.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other|      home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|Services|   Other|reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   Other|   Other|      home|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   Other| Teacher|      home|
|    GP|  M| 15|      U|    LE3|      A|   3|   2|Services|   Other|      home|
|    GP|  M| 15|      U|    GT3|      T|

In [15]:
df['Mjob'].apply(lambda x: str(x).capitalize())
df['Fjob'].apply(lambda x: str(x).capitalize())




0       Teacher
1         Other
2         Other
3      Services
4         Other
         ...   
390    Services
391    Services
392       Other
393       Other
394     At_home
Name: Fjob, Length: 395, dtype: object

### Step 7. Print the last elements of the data set.

In [22]:
spark.createDataFrame(students_subset.tail(10)).show()


+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    MS|  F| 18|      R|    GT3|      T|   2|   2| at_home|   other|     other|
|    MS|  F| 18|      R|    GT3|      T|   4|   4| teacher| at_home|reputation|
|    MS|  F| 19|      R|    GT3|      T|   2|   3|services|   other|    course|
|    MS|  F| 18|      U|    LE3|      T|   3|   1| teacher|services|    course|
|    MS|  F| 18|      U|    GT3|      T|   1|   1|   other|   other|    course|
|    MS|  M| 20|      U|    LE3|      A|   2|   2|services|services|    course|
|    MS|  M| 17|      U|    LE3|      T|   3|   1|services|services|    course|
|    MS|  M| 21|      R|    GT3|      T|   1|   1|   other|   other|    course|
|    MS|  M| 18|      R|    LE3|      T|   3|   2|services|   other|    course|
|    MS|  M| 19|      U|    LE3|      T|

In [20]:
df_subset.tail()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
390,MS,M,20,U,LE3,A,2,2,services,services,course,other
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother
392,MS,M,21,R,GT3,T,1,1,other,other,course,other
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother
394,MS,M,19,U,LE3,T,1,1,other,at_home,course,father


### Step 8. Did you notice the original dataframe is still lowercase? Why is that? Fix it and capitalize Mjob and Fjob.

In [29]:
students_subset = students_subset.withColumn("Mjob", capitalize_udf(students_subset["Mjob"]))
students_subset = students_subset.withColumn("Fjob", capitalize_udf(students_subset["Fjob"]))
students_subset.show()

+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|school|sex|age|address|famsize|Pstatus|Medu|Fedu|    Mjob|    Fjob|    reason|
+------+---+---+-------+-------+-------+----+----+--------+--------+----------+
|    GP|  F| 18|      U|    GT3|      A|   4|   4| At_home| Teacher|    course|
|    GP|  F| 17|      U|    GT3|      T|   1|   1| At_home|   Other|    course|
|    GP|  F| 15|      U|    LE3|      T|   1|   1| At_home|   Other|     other|
|    GP|  F| 15|      U|    GT3|      T|   4|   2|  Health|Services|      home|
|    GP|  F| 16|      U|    GT3|      T|   3|   3|   Other|   Other|      home|
|    GP|  M| 16|      U|    LE3|      T|   4|   3|Services|   Other|reputation|
|    GP|  M| 16|      U|    LE3|      T|   2|   2|   Other|   Other|      home|
|    GP|  F| 17|      U|    GT3|      A|   4|   4|   Other| Teacher|      home|
|    GP|  M| 15|      U|    LE3|      A|   3|   2|Services|   Other|      home|
|    GP|  M| 15|      U|    GT3|      T|

In [31]:
df_subset['Mjob'] = df_subset['Mjob'].apply(lambda x: str(x).capitalize())
df_subset['Fjob'] = df_subset['Fjob'].apply(lambda x: str(x).capitalize())

In [32]:
df_subset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian
0,GP,F,18,U,GT3,A,4,4,At_home,Teacher,course,mother
1,GP,F,17,U,GT3,T,1,1,At_home,Other,course,father
2,GP,F,15,U,LE3,T,1,1,At_home,Other,other,mother
3,GP,F,15,U,GT3,T,4,2,Health,Services,home,mother
4,GP,F,16,U,GT3,T,3,3,Other,Other,home,father
...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,Services,Services,course,other
391,MS,M,17,U,LE3,T,3,1,Services,Services,course,mother
392,MS,M,21,R,GT3,T,1,1,Other,Other,course,other
393,MS,M,18,R,LE3,T,3,2,Services,Other,course,mother


### Step 9. Create a function called majority that returns a boolean value to a new column called legal_drinker (Consider majority as older than 17 years old)

### Step 10. Multiply every number of the dataset by 10. 
##### I know this makes no sense, don't forget it is just an exercise